In [2]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="In the land of code, where algorithms reign,\nThere dwells a concept both clever and arcane.\nRecursion, a technique of elegance and power,\nUnraveling problems like a blooming flower.\n\nA function that calls itself without fear,\nIterating through loops, year after year.\nDivide and conquer, it loves to divide,\nSolving puzzles with grace and stride.\n\nLike a mirror reflecting its own image anew,\nRecursion looks back, finding what's true.\nBreaking down tasks into smaller bites,\nIt climbs the tree of recursion's heights.\n\nBase cases it seeks, to finally break free,\nFrom the endless cycle, from infinity.\nA dance of logic, a rhythm of code,\nIn the world of recursion, paths are boldly trod.\n\nSo remember this lesson, this poetic decree,\nRecursion's magic, for all to see.\nIn the realm of programming, where challenges loom,\nEmbrace the beauty of recursion's bloom.", role='assistant', function_call=None, tool_calls=None)


In [4]:
import openai
import os
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

In [25]:
def read_and_chunk_file(file_path, chunk_size=1024):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = read_and_chunk_file('sample3.txt')
for chunk in chunks:
    print(f"chunk: \n{chunk}")
    print("-------------------next chunk-------------------\n")

chunk: 
The future of technology is incredibly exciting, with advancements in artificial intelligence and machine learning leading the way. These technologies have the potential to revolutionize every aspect of our lives, from the way we work to how we interact with the world around us.

Artificial intelligence (AI) is rapidly evolving, enabling machines to learn from experience, adjust to new inputs, and perform human-like tasks. With the vast amounts of data being generated every day, AI algorithms are becoming increasingly sophisticated, making it possible to automate a wide range of activities, from driving cars to providing customer service.

Machine learning, a subset of AI, involves the development of algorithms that allow computers to learn and make decisions based on data. This technology is at the heart of many AI advancements, providing the foundation for systems that can improve their performance over time without being explicitly programmed.

In addition to technology, the

In [16]:
def embed_texts_with_openai(texts, model="text-embedding-3-small"):
    embeddings = []
    for text in texts:
        embeddings.append(client.embeddings.create(input=text, model=model).data[0].embedding)
    return embeddings

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_cosine_similarity(embedding, embeddings):
    similarities = cosine_similarity([embedding], embeddings)
    return similarities[0]

In [8]:
def recommend_text_chunks(file_path, input_query, top_n=5):
    chunks = read_and_chunk_file(file_path)
    chunk_embeddings = embed_texts_with_openai(chunks)
    query_embedding = embed_texts_with_openai([input_query])[0]
    
    similarities = calculate_cosine_similarity(query_embedding, chunk_embeddings)
    most_relevant_indices = np.argsort(similarities)[-top_n:][::-1]
    
    return [(chunks[i], similarities[i]) for i in most_relevant_indices]

In [27]:
file_path = 'sample3.txt'
input_query = "Sustainablity is the key to the future."
top_n = 5  # Number of recommendations you want


try:
    recommendations = recommend_text_chunks(file_path, input_query, top_n)
except Exception as e:
    print(f"An error occurred: {e.message}")

for chunk, similarity in recommendations:
    print(f"Chunk: {chunk}\nSimilarity: {similarity}\n")
    print("--------------------------------------------------\n")

Chunk: ossil fuels, decrease greenhouse gas emissions, and move towards a more sustainable and clean energy future.

In conclusion, the intersection of technology, nature, and sustainability holds the key to solving many of the world's most pressing challenges. By embracing innovation and respecting the natural world, we can create a brighter future for all.
Similarity: 0.551931026654206

--------------------------------------------------

Chunk: tural world continue to inspire and amaze us. From the intricate ecosystems of rainforests to the vast expanses of the ocean, nature is a constant reminder of the wonders that surround us.

Conservation efforts are crucial in protecting these natural wonders for future generations. By understanding the delicate balance of ecosystems and the impact of human activity, we can work towards a more sustainable future, preserving the planet's biodiversity and natural resources.

The exploration of space represents one of humanity's most ambitious end